In [1]:
import cv2
import os
import numpy as np
from UNet import UNet
from sklearn.model_selection import train_test_split
from skimage.restoration import denoise_nl_means, estimate_sigma

import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from sklearn.model_selection import train_test_split


2024-05-16 21:39:34.600646: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def leer_imagenes(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [3]:
def convertir_a_grises(images):
    gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
    return gray_images


In [4]:
from skimage.restoration import denoise_nl_means, estimate_sigma

def aplicar_filtro_anisotropico(images):
    filtered_images = []
    for image in images:
        sigma_est = np.mean(estimate_sigma(image))
        denoised = denoise_nl_means(image, h=1.15 * sigma_est, fast_mode=True,
                                    patch_size=5, patch_distance=6)
        filtered_images.append(denoised)
    return filtered_images



In [5]:
def extract_patches(images, filtered_images, patch_size=32, num_patches=500000):
    xi, yi = [], []
    total_patches_per_image = num_patches // len(images)
    
    for img, filt_img in zip(images, filtered_images):
        count = 0
        while count < total_patches_per_image:
            i = np.random.randint(0, img.shape[0] - patch_size)
            j = np.random.randint(0, img.shape[1] - patch_size)
            xi.append(img[i:i+patch_size, j:j+patch_size])
            yi.append(filt_img[i:i+patch_size, j:j+patch_size])
            count += 1
    
    return np.array(xi), np.array(yi)

In [6]:
def main(folder='test', k=32):
    images = leer_imagenes(folder)
    gray_images = convertir_a_grises(images)
    filtered_images = aplicar_filtro_anisotropico(gray_images)
    xi, yi = extract_patches(gray_images, filtered_images, patch_size=32)

    return xi, yi

# Ejecutar la función main
xi, yi  = main()
xi_train, xi_test, yi_train, yi_test = train_test_split(xi, yi, test_size=0.2)

print(f"xi shape: {xi.shape}, yi shape: {yi.shape}")
print(f"Train shapes - xi: {xi_train.shape}, yi: {yi_train.shape}")
print(f"Test shapes - xi: {xi_test.shape}, yi: {yi_test.shape}")

def unet_model(input_size=(32, 32, 1)):
    inputs = Input(input_size)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    up1 = UpSampling2D(size=(2, 2))(conv2)
    merge1 = concatenate([conv1, up1], axis=3)
    conv3 = Conv2D(32, 3, activation='relu', padding='same')(merge1)
    conv4 = Conv2D(1, 1, activation='sigmoid')(conv3)
    model = Model(inputs=inputs, outputs=conv4)
    return model

model = unet_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(xi_train.reshape(-1, 32, 32, 1), yi_train.reshape(-1, 32, 32, 1), epochs=15, batch_size=64, validation_split=0.1)




xi shape: (500000, 32, 32), yi shape: (500000, 32, 32)
Train shapes - xi: (400000, 32, 32), yi: (400000, 32, 32)
Test shapes - xi: (100000, 32, 32), yi: (100000, 32, 32)


In [ ]:
# input_shape = (32, 32, 1)
# unet = UNet()
# model = unet.build_model(input_shape)


In [ ]:
# history = model.fit(X_train, epochs=20, validation_data=x_test)